<a href="https://colab.research.google.com/github/Rpmalukani22/Sentiment-analysis-on-IMDB-Movie-Reviews-using-Bert/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 1.4MB/s 
     |████████████████████████████████| 983kB 48.4MB/s 
     |████████████████████████████████| 266kB 41.9MB/s 
     |████████████████████████████████| 1.3MB 46.3MB/s 
     |████████████████████████████████| 1.1MB 41.2MB/s 
     |████████████████████████████████| 471kB 43.2MB/s 
     |████████████████████████████████| 2.9MB 53.6MB/s 
     |████████████████████████████████| 890kB 50.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.1-cp36-none-any.whl size=25276126 sha256=dd846d87e567f9e8bccdd1a3332a5fbb1281178a991bd22e962c09da9a67afc8
  Stored in directory: /root/.cache/pip/wheels/4c/af/2a/3b4aa379166bc910a4132ecb83f795c854b2a4293f83f0b736
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=74aeb23fd66a0fc192da6744a578bfb58ed380a58444710d566c03550226fe90
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for syn

In [ ]:
# import tensorflow as tf
# print(tf.__version__)

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [ ]:
df_train = pd.read_excel('./IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)
df_test = pd.read_excel('./IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype = str)

In [ ]:
df_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [ ]:
df_train.tail()

,Reviews,Sentiment
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Reviews    25000 non-null  object
 1   Sentiment  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [ ]:
df_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [ ]:
(X_train,y_train),(X_test,y_test),prepoc = text.texts_from_df(train_df = df_train,
                                                              text_column = 'Reviews',
                                                              label_columns = ['Sentiment'],
                                                              val_df = df_test,
                                                              maxlen = 500,
                                                              preprocess_mode = 'bert') 

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert',train_data = (X_train,y_train),preproc = prepoc)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner = ktrain.get_learner(model,train_data = (X_train,y_train),val_data=(X_test,y_test),batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 5585s 1s/step - loss: 0.2528 - accuracy: 0.8933 - val_loss: 0.1015 - val_accuracy: 0.9676


In [ ]:
predictor = ktrain.get_predictor(learner.model,prepoc)


In [ ]:
predictor.save('/content/drive/My Drive/bert')

In [ ]:
data = ['awesome movie it was! What an acting.','I wasted my money!','Acting was ok not that great']

In [ ]:
predictor.predict(data)

['pos', 'neg', 'neg']

In [ ]:
predictor.predict(data,return_proba=True)

array([[0.00320592, 0.99679404],
       [0.9199904 , 0.08000954],
       [0.8757579 , 0.12424216]], dtype=float32)